In [13]:
import duckdb

con = duckdb.connect("../db/synpuf.duckdb")

# Stream through the carrier table in manageable chunks
chunk_size = 100_000
offset = 0

while True:
    query = f"SELECT * FROM carrier LIMIT {chunk_size} OFFSET {offset}"
    df_chunk = con.execute(query).df()
    if df_chunk.empty:
        break
    
    print(f"Loaded rows {offset} - {offset + len(df_chunk)}")
    # process df_chunk here (EDA, feature extraction, save to parquet, etc.)
    
    offset += chunk_size


Loaded rows 0 - 100000
Loaded rows 100000 - 200000
Loaded rows 200000 - 300000
Loaded rows 300000 - 400000
Loaded rows 400000 - 500000
Loaded rows 500000 - 600000
Loaded rows 600000 - 700000
Loaded rows 700000 - 800000
Loaded rows 800000 - 900000
Loaded rows 900000 - 1000000
Loaded rows 1000000 - 1100000
Loaded rows 1100000 - 1200000
Loaded rows 1200000 - 1300000
Loaded rows 1300000 - 1400000
Loaded rows 1400000 - 1500000
Loaded rows 1500000 - 1600000
Loaded rows 1600000 - 1700000
Loaded rows 1700000 - 1800000
Loaded rows 1800000 - 1900000
Loaded rows 1900000 - 2000000
Loaded rows 2000000 - 2100000
Loaded rows 2100000 - 2200000
Loaded rows 2200000 - 2300000
Loaded rows 2300000 - 2400000
Loaded rows 2400000 - 2500000
Loaded rows 2500000 - 2600000
Loaded rows 2600000 - 2700000
Loaded rows 2700000 - 2800000
Loaded rows 2800000 - 2900000
Loaded rows 2900000 - 3000000
Loaded rows 3000000 - 3100000
Loaded rows 3100000 - 3200000
Loaded rows 3200000 - 3300000
Loaded rows 3300000 - 3400000
Load

In [9]:
df_chunk.head()

,DESYNPUF_ID,CLM_ID,CLM_FROM_DT,CLM_THRU_DT,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2,ICD9_DGNS_CD_3,ICD9_DGNS_CD_4,ICD9_DGNS_CD_5,ICD9_DGNS_CD_6,...,LINE_ICD9_DGNS_CD_4,LINE_ICD9_DGNS_CD_5,LINE_ICD9_DGNS_CD_6,LINE_ICD9_DGNS_CD_7,LINE_ICD9_DGNS_CD_8,LINE_ICD9_DGNS_CD_9,LINE_ICD9_DGNS_CD_10,LINE_ICD9_DGNS_CD_11,LINE_ICD9_DGNS_CD_12,LINE_ICD9_DGNS_CD_13
